In [1]:
from utils import *
import os, glob
import numpy as np
import dicom as di
import nibabel as ni
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

# 3D Affine Formulae
---
\begin{align}
\mathbf{A}_{multi} = \begin{vmatrix}
F_{11}\Delta_r & F_{12}\Delta_c & \frac{T_1^1-T_1^N}{1-N} & T_1^1 \\
F_{21}\Delta_r & F_{22}\Delta_c & \frac{T_2^1-T_2^N}{1-N} & T_2^1 \\
F_{31}\Delta_r & F_{32}\Delta_c & \frac{T_3^1-T_3^N}{1-N} & T_3^1 \\
0 & 0 & 0 & 1
\end{vmatrix}
\end{align}

- $F_{ij}$ : ImageOrientationPatient 정보 
    - ($i_1$, $i_2$, $i_3$, $i_4$, $i_5$, $i_6$) = ($F_{12}$, $F_{22}$, $F_{32}$, $F_{11}$, $F_{21}$, $F_{31}$)  
    
- $\Delta_r$ : Row spacing  

- $\Delta_c$ : Col spacing  

- $T_1$ : 첫번째 DICOM 의 ImagePositionPatient 정보
    - (X, Y, Z) 형식  

- $T_N$ : 마지막 DICOM 의 ImagePositionPatient 정보
    - (X, Y, Z) 형식  
    
- $1-N$ : 1 - DICOM 갯수  



- $(X_{csv}, Y_{csv}, Z_{csv}, 1) = A_{multi} * (X,Y,Z,1)$  

- labeling 할때는 $(X,Y,Z,1) = inv(A_{multi}) * (X_{csv}, Y_{csv}, Z_{csv}, 1)$ 해줘야함.

In [5]:
to_path = './data/'

num_path = os.listdir(to_path)
num_path.remove('.ipynb_checkpoints')
num_path.remove('data_2d.npz')
num_path.remove('data_3d.npz')
num_path.sort()

data_img = {}
data_lab = {}
for num in num_path:
    data_path = os.listdir(to_path+num)
    
    #print(data_path)
    for idx, data in enumerate(data_path):
        
        if data in ['12692', '11961'] : continue
            
        img, ground, ori_coor = make_data(to_path, num, data, use_scale=False)

        img = resizing_data(img)
        #img = np.reshape(img, [1,-1,256,256])
        
        ground = resizing_data(ground)
        #ground = np.reshape(ground, [1,-1,256,256])
        
        if num =='01' and idx ==0:
            data_img = img
            data_lab = ground
        else:
            data_img = np.concatenate((data_img, img), axis=0)
            data_lab = np.concatenate((data_lab, ground), axis=0)
        print("Num : %s , Data : %s , Image shape : %s"%(num,data,img.shape))
    print("----------------------------------------------------------------")
        

Num : 01 , Data : 12653 , Image shape : (170, 256, 256)
Num : 01 , Data : 12841 , Image shape : (156, 256, 256)
Num : 01 , Data : 12575 , Image shape : (171, 256, 256)
Num : 01 , Data : 12803 , Image shape : (160, 256, 256)
----------------------------------------------------------------
Num : 02 , Data : 12127 , Image shape : (157, 256, 256)
Num : 02 , Data : 11439 , Image shape : (186, 256, 256)
Num : 02 , Data : 12459 , Image shape : (164, 256, 256)
Num : 02 , Data : 5569 , Image shape : (153, 256, 256)
Num : 02 , Data : 11900 , Image shape : (176, 256, 256)
Num : 02 , Data : 12280 , Image shape : (140, 256, 256)
Num : 02 , Data : 12422 , Image shape : (157, 256, 256)
Num : 02 , Data : 11906 , Image shape : (151, 256, 256)
----------------------------------------------------------------
Num : 03 , Data : 11760 , Image shape : (184, 256, 256)
Num : 03 , Data : 7360 , Image shape : (158, 256, 256)
Num : 03 , Data : 11534 , Image shape : (176, 256, 256)
Num : 03 , Data : 11498 , Image 

In [9]:
np.savez('/mnt/d/tumor/01', img=data_img, lab = data_lab)

# 01
---
12575 : 171  
12653 : 170  
12692 : 156  
12803 : 160  
12841 : 156  

# 02
---
11439 : :187.dcm   (:186) 186  
11900 : 84.dcm:    (83:)  175  
11906 : :152.dcm   (:151) 151  
11961 : 25.dcm:    (24:)  167  
12127 : 25.dcm:    (24:)  156  
12280 : 25.dcm:    (24:)  139  
12422 : 25.dcm:    (24:)  156  
12459 : 25.dcm:    (24:)  163  
 5569 : 25.dcm:    (24:)  152  

# 03
---
11498 : 25.dcm:205.dcm (24:204)  180  
11534 : 89.dcm:        (88:)     175  
11760 : :185.dcm       (:184)    184  
12259 : 25.dcm:203.dcm (24:202)  178  
12735 : 25.dcm:        (24:)     154  
 6002 : 61.dcm:203.dcm (60:202)  142  
 6702 : :161.dcm       (:160)    160  
 7360 : 25.dcm:        (24:)     157  

# MAX, MIN
---

- max : 186
- min : 139
- mean : 160